In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'Control': 0, 'Stimulated': 1}

# Load graph dataset and process if neede
data_dir = (Path().cwd().parents[0] / 'data'/ 'pixelgen').absolute() / 'Tcell' / 'graphs'

dataset = PPIGraph.GraphDataset(data_dir, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(932):
Number of graphs: 932
Number of features: 80
Number of classes: 2
Train set: 448, test set: 372, val set: 112
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 318876], pos=[53210, 3], labels=[53210, 80], weight=[318876], condition=[32], id=[32], train_mask=[53210], test_mask=[53210], x=[53210, 80], y=[32], edge_weight=[318876], name=[32], batch=[53210], ptr=[33])



In [9]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(932):
Number of graphs: 932
Number of features: 80
Number of classes: 2
Train set: 448, test set: 372, val set: 112
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 337386], pos=[56295, 3], labels=[56295, 80], weight=[337386], condition=[32], id=[32], train_mask=[56295], test_mask=[56295], x=[56295, 80], y=[32], edge_weight=[337386], name=[32], batch=[56295], ptr=[33])



# K-fold on filter dataset

In [10]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [12]:
# Define condition mapping
condition_mapping = {'Control': 0, 'Stimulated': 1}

# Load graph dataset and process if neede
data_dir = (Path().cwd().parents[0] / 'data'/ 'pixelgen').absolute() / 'Tcell' / 'graphs'

dataset = PPIGraph.GraphDataset(data_dir, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [13]:
len(dataset)

932

In [14]:
dataset_name = 'pixelgen'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / 'pixelgen' / 'Tcell' /"saved_models" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01122024_{dataset_name}_{condition}'

In [15]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)
epochs = 65

# NUM_LAYERS = 2
# HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']
# num_layers = [2,3,4]
# hiddens = [16, 32, 64]


# # model = 'GAT'
# model = 'GINConv'
params = [[3,128,'attention','MLP'], 
[3,128,'attention','GraphConv'],
[3,128,'attention2','GCN'],
[3,128,'attention','GAT_norm'],
[3,128,'mean','GINConv_norm']]

Global seed set to 42


In [16]:
params

[[3, 128, 'attention', 'MLP'],
 [3, 128, 'attention', 'GraphConv'],
 [3, 128, 'attention2', 'GCN'],
 [3, 128, 'attention', 'GAT_norm'],
 [3, 128, 'mean', 'GINConv_norm']]

In [17]:
torch.set_float32_matmul_precision('medium')

In [18]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset)):
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in params:
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=80,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                embedding=False,
                                                batch_size=32,
                                                graph_pooling=pool)
        run.finish()


y:\coskun-lab\Thomas\23_PLA_revision\data\pixelgen\Tcell\saved_models\pixelgen\Graph_GNNs_Kfold\MLP_3_128_onehot_0\attention\GraphLevelMLP\GraphLevelMLP.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\pixelgen\Tcell\saved_models\pixelgen\Graph_GNNs_Kfold\GraphConv_3_128_onehot_0\attention\GraphLevelGraphConv\GraphLevelGraphConv.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\pixelgen\Tcell\saved_models\pixelgen\Graph_GNNs_Kfold\GCN_3_128_onehot_0\attention2\GraphLevelGCN\GraphLevelGCN.ckpt


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\pytorch_lightning\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Using GAT


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\pixelgen\Tcell\saved_models\pixelgen\Graph_GNNs_Kfold\GAT_norm_3_128_onehot_0\attention\GraphLevelGAT_norm exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | GAT              | 121 K 
2  | head        | Sequential       | 8.4 K 
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryAccuracy   | 0     
5  | train_auroc | BinaryAUROC      | 0     
6  | train_f1    | BinaryF1Score    | 0     
7  | valid_acc   | BinaryAccuracy   | 0     
8  | valid_auroc | BinaryAUROC      | 0     
9  | valid_f1 

...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▇▇▇▇▇▇▆▇▇██▇█▇██▇▇▇▇█▇█▇▆▇▇▇████▇███
train_auc,▁▆▇▇█▇███▇▇█████████████████▇███████████
train_f1,▁▅▆▇▇▇▇▇▇▇▆▇▇██▇█▇██▇▇▇▇█▇█▇▆▇▇▇████▇███
train_loss_epoch,█▅▃▂▂▂▂▂▅▃▃▂▂▁▁▂▁▂▁▁▂▂▂▁▁▂▁▄▄▂▂▂▁▁▁▁▂▁▁▁
train_loss_step,▄█▂▁▁▃▂▂▁▄▁▁▃▁▅▂▂▂▁▁▂▁▁▁▂▁▁▃▂▁▂▁▁▂▁▁▁▁▂▂
trainer/global_step,▁▁▁▁▂▁▁▂▁▁▃▃▂▂▄▄▂▄▄▂▅▅▂▅▆▂▂▆▂▂▂▇▂▃▇▃▃█▃▃
val_acc,▇▇▇██▇▇▇█▂▆▄▆▃▅▅▃▆▂▄▄█▆▇▇▆▇▁▄▇█▇▆▇▇█▅█▆▇
val_auc,▆▄▆▇█▆█▃▇█▅▃▇▅▄▄▃▄▃▄▇▇▇▅▂▁▇▇█▇█▇▇▇▇▇▅▇▁▅
val_f1,▇▇█████▇█▅▇▆▇▆▆▆▆▇▅▆▅█▇▇▇▆█▁▅▇█▇▇███▆█▇▇


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\pixelgen\Tcell\saved_models\pixelgen\Graph_GNNs_Kfold\GINConv_norm_3_128_onehot_4\mean\GraphLevelGINConv_norm exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | GIN              | 94.2 K
2 | head        | Sequential       | 8.4 K 
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 

GINConv


`Trainer.fit` stopped: `max_epochs=65` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▇▇▇▇▇▇▆▇▇███▇███████████▇▇██▇████▇█████
train_auc,▁▇█████▇█████▇██████████████████████████
train_f1,▁▇▇▇▇▇▇▆▇▇███▇███████████▇▇██▇████▇█████
train_loss_epoch,█▃▃▂▂▂▂▃▂▂▂▁▁▃▂▂▂▂▁▁▁▁▁▁▂▂▂▂▁▂▁▁▁▂▃▁▁▁▂▁
train_loss_step,▄▂▁▁▂▂▃▁▂▂▁▁▁▄▂▄▁▁▁▁▁▁▁▁█▂▁▁▁▁▁▁▁█▁▁▁▅▁▁
trainer/global_step,▁▁▁▁▂▁▁▂▁▁▃▃▂▂▄▄▂▄▄▂▅▅▂▅▆▂▂▆▂▂▂▇▂▃▇▃▃█▃▃
val_acc,▁▄▅▄▆▇▇▃▅▆▄▇▆█▅▆▄▇▆▅▆▇▃▇▅▆▇▅▆▆▄▆▇▆▅▂█▅▇▇
val_auc,▅▆█▁▅██▃▅▅▇▇███▇▁█▇█▆█▂█▃██▇▇▇▃██▆▅▃█▆█▇
val_f1,▁▄▅▃▆▇▇▃▅▆▄▇▆█▄▆▃▇▆▄▆▇▂▇▅▆▇▄▆▆▄▆▇▆▅▂█▅▇▇
